# SPD

Replicate the design of [Smaldino et al. (2013)](https://www.journals.uchicago.edu/doi/10.1086/669615).

Dummy agents - no interactions just movement.

## Imports & properties

In [ ]:
# model
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

# visualization
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import holoviews as hv
%load_ext holoviews.ipython
import seaborn as sns
sns.set_theme(style="darkgrid")

# parameter sweep
from mesa.batchrunner import BatchRunner 

In [ ]:
# environment properties

grid_size       = 10
N               = 1
starting_energy = 10
living_cost     = 1

max_steps       = 10e6

## Setup model

In [ ]:
# errors

class ModelError(Exception):
    pass
        
class UnidentifiedCellError(ModelError):
    pass


In [ ]:
class DummyAgent(Agent):
    """
    Always Abstain strategy
    - never interacts with any other agents
    - moves around randomly
    - dies when energy depleted
    """
    
    def __init__(self, model, energy=starting_energy):
        super().__init__(model.next_id(), model)
        self.energy = energy
        
        
    def step(self):
        # pay cost of living
        self.energy -= living_cost
        if self.energy <= 0:
            # agent died
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
            
        # alive
        self.model.n_agents += 1
        
        # move to a random adjacent unoccupied square if exists
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
        neighborhood = filter(lambda c: self.model.grid.is_cell_empty(c), neighborhood)
        neighborhood = sorted(list(neighborhood))      
        if neighborhood:
            cell = self.random.choice(neighborhood)
            self.model.grid.move_agent(self, cell)
        

In [ ]:
class SPDModel(Model):
    
    def __init__(self, n0=N, grid_size=grid_size, wrap=True):
        """
        Args:
            n0:         starting number of agents
            grid_size:  size length of square grid to use
            wrap:       whether to wrap grid
        """
        super().__init__()
        
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(grid_size, grid_size, torus=wrap)
        
        # Setup agents
        for i in range(n0):
            agent = DummyAgent(self)
            self.grid.position_agent(agent)
            self.schedule.add(agent)
        
        self.n_agents = n0
        
        # Init model
        self.running = True
        
        self.datacollector = DataCollector(
            {
                "n_agents": "n_agents",
            },
            {
                "x": lambda a: a.pos[0],
                "y": lambda a: a.pos[1],
            },
        )
        self.datacollector.collect(self)
        
        
    def step(self):
        
        # reset model counters
        self.n_agents = 0
        
        self.schedule.step()
        self.datacollector.collect(self)
        
        # stop the model if no agents are alive
        if self.n_agents == 0:
            self.running = False
        

## Run model

In [ ]:
spd = SPDModel()

def value(cell):
    if cell is None:
        return 0
    elif isinstance(cell, Agent):
        return 1
    else:
        raise UnidentifiedCellError()
        
hmap = hv.HoloMap(kdims='step')
i = 0
while spd.running:
    spd.step()
    data = np.array([[value(c) for c in row] for row in spd.grid.grid])
    hmap[i] = hv.Image(data, vdims=[hv.Dimension('State', range=(0,3))])
    i += 1
hmap

In [ ]:
results = spd.datacollector.get_model_vars_dataframe()

sns.lineplot(data=results)

## Paramater sweep

In [ ]:
variable_params = {
    "n0": range(1,100,1),
}
fixed_params = {
    "grid_size": grid_size,
    "wrap":      True,
}

param_run = BatchRunner(SPDModel,
                        variable_params,
                        fixed_params,
                        max_steps=max_steps,
                        model_reporters={
                            "n_agents": lambda m: m.n_agents,
                        })

param_run.run_all()

In [ ]:
run_data = param_run.get_model_vars_dataframe()

sns.scatterplot(x="n0", y="n_agents", data=run_data)